In [19]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
w = cluster.start_worker(ncores=2)

In [20]:
import re

import dask.dataframe as dd
import pandas as pd

raw = pd.read_table('/Users/minhmai/Downloads/train.txt', delimiter=' ', header=None)
raw.columns = ['word', 'pos', 'wsj']
raw.head(50)

,word,pos,wsj
0,Confidence,NN,B-NP
1,in,IN,B-PP
2,the,DT,B-NP
3,pound,NN,I-NP
4,is,VBZ,B-VP
5,widely,RB,I-VP
6,expected,VBN,I-VP
7,to,TO,I-VP
8,take,VB,I-VP
9,another,DT,B-NP


In [21]:
def refactor_pos(x):
    # helps deal with imbalances between classes
    if x == '.':
        return 'stopper'
    elif x.startswith('V') or x.startswith('RB') or x == 'JJ':
        return 'verb/adverb/adjective'
    elif x in ['NS', 'NNS', 'NN']:
        return 'noun'
    elif x in ['IN', 'TO', 'DT']:
        return 'preposition'
    else:
        return 'other'

# remove symbols
def preprocess_dataframe(data):
    regex = re.compile(r'[^.a-z0-9]')
    data['is_symbol'] = data.pos.apply(lambda x: True if regex.match(x) else False)
    data = data[((data.is_symbol == True) | (data.pos == '.'))]

    # refactor pos tagging
    data['new_pos'] = data.pos.apply(lambda x: refactor_pos(x))
    data = data[['word', 'new_pos']]
    data['word'] = data.word.apply(lambda x: x.lower())
    corpus = {k: v for k, v in zip(data['word'].unique(), range(data['word'].nunique())) if k != '.'}
    corpus['.'] = 999999
    data['word'] = data.word.apply(lambda x: corpus[x])
    return data


In [74]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

cleaned = preprocess_dataframe(raw)
data = cleaned.loc[cleaned['word'] != 999999, :]
X_train, X_test, y_train, y_test = train_test_split(
    data['word'],
    data['new_pos'],
    test_size=0.2,
    random_state=5
)

model = GaussianNB()
model.fit(X_train.reshape(-1, 1), y_train)
preds = model.predict(X_test.reshape(-1, 1))
# validation set metrics
print(classification_report(y_test, preds))


/Users/minhmai/Envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
/Users/minhmai/Envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


                       precision    recall  f1-score   support

                 noun       0.29      0.10      0.15      8747
                other       0.25      0.23      0.24     12930
          preposition       0.36      0.98      0.52      9177
              stopper       0.00      0.00      0.00        19
verb/adverb/adjective       0.00      0.00      0.00      9728

          avg / total       0.22      0.32      0.23     40601



/Users/minhmai/Envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
test = pd.read_table('/Users/minhmai/Downloads/test.txt', delimiter=' ', header=None)
test.columns = ['word', 'pos', 'wsj']
test_data = preprocess_dataframe(test)
test_preds = model.predict(test_data['word'].reshape(-1, 1))
# test set metrics
print(classification_report(test_data['new_pos'], test_preds))

/Users/minhmai/Envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/minhmai/Envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


                       precision    recall  f1-score   support

                 noun       0.30      0.08      0.13      9676
                other       0.45      0.00      0.00     10971
          preposition       0.41      0.96      0.57     10269
verb/adverb/adjective       0.34      0.46      0.39     10670

          avg / total       0.38      0.38      0.27     41586



In [27]:
confusion_matrix(test_data['new_pos'], test_preds)

array([[ 777,   15, 3307, 5577],
       [ 976,   25, 6268, 3702],
       [   3,    0, 9901,  365],
       [ 807,   15, 4916, 4932]])

In [32]:
data.head(30)

,word,new_pos
0,0,noun
1,1,preposition
2,2,preposition
3,3,noun
4,4,verb/adverb/adjective
5,5,verb/adverb/adjective
6,6,verb/adverb/adjective
7,7,preposition
8,8,verb/adverb/adjective
9,9,preposition


distributed.core - WARNING - Event loop was unresponsive in Scheduler for 1846.82s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 1846.88s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 1846.89s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 1846.89s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - WARNING - Event loop was unresponsive in Nanny for 1846.89s.  This is often caused by long-running GIL-holding functions or m

# Trying to build a GRU

In [1]:
# feedforward is a loop until the end of sentence
# predict will occur at every step but has memory of previous inputs
# reprocess to mark end of sentence
# at end of sentence, clear "memory" (find function)
# all is it learn is when to open/close gate and let it affect state and update state
import tensorflow as tf 

data = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test
}

In [25]:
?tf.reduce_mean

In [18]:
?tf.zeros

In [14]:
var

<tf.distributions.Normal 'Normal' batch_shape=() event_shape=() dtype=float32>